<a href="https://colab.research.google.com/github/rpasquini/defidata/blob/main/AAVE_Unhealthy_positions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unhealthy positions Monitor

In [1]:
import requests
from pprint import pprint
from datetime import datetime
import pandas as pd

In [2]:
#(See https://thegraph.com/hosted-service/subgraph/aave/protocol-v2?version=current)
subgraph_http_address='https://api.thegraph.com/subgraphs/name/aave/protocol-v2'

# function to use requests.post to make an API call to the subgraph url
def run_query(query):

    # endpoint where you are making the request
    request = requests.post(subgraph_http_address,
                            json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed. return code is {}.      {}'.format(request.status_code, query))

In [7]:
#users(first:1000, skip: """+str(1000*count)+""", orderBy: id, orderDirection: desc, where: {borrowedReservesCount_gt: 0}) {
count=0
first=10
query="""{
users(first:"""+str(first)+""", skip: """+str(1000*count)+""", orderBy: id, orderDirection: desc, where: {borrowedReservesCount_gt: 0}){
          id
          borrowedReservesCount
          collateralReserve:reserves(where: {currentATokenBalance_gt: 0}) {
            currentATokenBalance
            reserve{
              usageAsCollateralEnabled
              reserveLiquidationThreshold
              reserveLiquidationBonus
              borrowingEnabled
              utilizationRate
              symbol
              underlyingAsset
              decimals
              price {
                priceInEth
              }

            }
          }
          borrowReserve: reserves(where: {currentTotalDebt_gt: 0}) {
            currentTotalDebt
            reserve{
              usageAsCollateralEnabled
              reserveLiquidationThreshold
              borrowingEnabled
              utilizationRate
              symbol
              underlyingAsset
              decimals
              price {
                priceInEth
              }

            }
          }
      }
      }
"""
result = run_query(query)
result

{'data': {'users': [{'id': '0xffff8941130157a0153fb5be2618b257f28d3b55',
    'borrowedReservesCount': 1,
    'collateralReserve': [{'currentATokenBalance': '385716688415711350',
      'reserve': {'usageAsCollateralEnabled': True,
       'reserveLiquidationThreshold': '8600',
       'reserveLiquidationBonus': '10500',
       'borrowingEnabled': True,
       'utilizationRate': '0.52800523',
       'symbol': 'WETH',
       'underlyingAsset': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
       'decimals': 18,
       'price': {'priceInEth': '1000000000000000000'}}}],
    'borrowReserve': [{'currentTotalDebt': '353684533943004137474',
      'reserve': {'usageAsCollateralEnabled': True,
       'reserveLiquidationThreshold': '8700',
       'borrowingEnabled': True,
       'utilizationRate': '0.31028983',
       'symbol': 'DAI',
       'underlyingAsset': '0x6b175474e89094c44da98b954eedeac495271d0f',
       'decimals': 18,
       'price': {'priceInEth': '836885880000000'}}}]},
   {'id': '0xfffc

In [ ]:
userdata0=result['data']['users'][0]
userdata0

{'id': '0xffff8941130157a0153fb5be2618b257f28d3b55',
 'borrowedReservesCount': 1,
 'collateralReserve': [{'currentATokenBalance': '385716688415711350',
   'reserve': {'usageAsCollateralEnabled': True,
    'reserveLiquidationThreshold': '8600',
    'reserveLiquidationBonus': '10500',
    'borrowingEnabled': True,
    'utilizationRate': '0.53359029',
    'symbol': 'WETH',
    'underlyingAsset': '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
    'price': {'priceInEth': '1000000000000000000'},
    'decimals': 18}}],
 'borrowReserve': [{'currentTotalDebt': '353684533943004137474',
   'reserve': {'usageAsCollateralEnabled': True,
    'reserveLiquidationThreshold': '8700',
    'borrowingEnabled': True,
    'utilizationRate': '0.31309154',
    'symbol': 'DAI',
    'underlyingAsset': '0x6b175474e89094c44da98b954eedeac495271d0f',
    'price': {'priceInEth': '841484886277113'},
    'decimals': 18}}]}

In [ ]:
385716688415711350*(10**-18)

0.3857166884157114

In [ ]:
def test(userdata):

    """Computes total collateral times threshold -across all reserves/currencies - (in ETH)
    """
    total_collateral=0
    total_collateral_threshold=0

    for collateral_reserve in userdata["collateralReserve"]:
        decimals=int(collateral_reserve["reserve"]["decimals"])
        print('decimals',decimals)
        price_in_eth = int(collateral_reserve["reserve"]["price"]["priceInEth"])/ (10 ** decimals)
        print('price in eth',price_in_eth)
        current_a_token_balance = int(collateral_reserve["currentATokenBalance"])/ (10 ** decimals)
        print('current_a_token_balance',current_a_token_balance)
        total_collateral +=price_in_eth * current_a_token_balance
        print('total_collateral',total_collateral)
        liquidation_threshold=(int(collateral_reserve['reserve']['reserveLiquidationThreshold']) / 10000)
        print('liquidation_threshold',liquidation_threshold)

        #sum of collateral times thresholds 
        total_collateral_threshold += price_in_eth * current_a_token_balance * liquidation_threshold
        print('total_collateral_threshold',total_collateral_threshold)
            

    return total_collateral_threshold

test(userdata0)

decimals 18
price in eth 1.0
current_a_token_balance 0.38571668841571133
total_collateral 0.38571668841571133
liquidation_threshold 0.86
total_collateral_threshold 0.33171635203751176


0.33171635203751176

In [ ]:
def compute_total_borrowed(userdata):

  """Computes total borrowed  -across all reserves/currencies - (in ETH)
  """
  total_borrowed = 0
  # Three additional variables to detect the largest borrow across all currencies.
  #max_borrowed_symbol = None
  #max_borrowed_principal = 0
  #max_borrowed_price_in_eth = 0

  for borrow_reserve in userdata["borrowReserve"]:
            decimals=int(borrow_reserve["reserve"]["decimals"])
            price_in_eth = int(borrow_reserve["reserve"]["price"]["priceInEth"]) / (10 ** 18 )
            principal_borrowed = int(borrow_reserve["currentTotalDebt"]) / (10 **decimals )
            total_borrowed += price_in_eth * principal_borrowed
            
            #if principal_borrowed > max_borrowed_principal:
                #max_borrowed_symbol = borrow_reserve["reserve"]["symbol"]
                #max_borrowed_principal = principal_borrowed
                #max_borrowed_price_in_eth = price_in_eth
  
  return  total_borrowed

def compute_total_collateral_times_thresholds(userdata, print=False):

    """Computes total collateral times threshold -across all reserves/currencies - (in ETH)
    """

    """Computes total collateral times threshold -across all reserves/currencies - (in ETH)
    """
    total_collateral=0
    total_collateral_threshold=0

    for collateral_reserve in userdata["collateralReserve"]:
        decimals=int(collateral_reserve["reserve"]["decimals"])

        price_in_eth = int(collateral_reserve["reserve"]["price"]["priceInEth"])/ (10 ** 18)

        current_a_token_balance = int(collateral_reserve["currentATokenBalance"])/ (10 ** decimals)

        total_collateral +=price_in_eth * current_a_token_balance

        liquidation_threshold=(int(collateral_reserve['reserve']['reserveLiquidationThreshold']) / 10000)


        #sum of collateral times thresholds 
        total_collateral_threshold += price_in_eth * current_a_token_balance * liquidation_threshold


        if print: 
          print('decimals',decimals)
          print('price in eth',price_in_eth)
          print('current_a_token_balance',current_a_token_balance)
          print('total_collateral',total_collateral)
          print('liquidation_threshold',liquidation_threshold)
          print('total_collateral_threshold',total_collateral_threshold)

    return total_collateral_threshold

def compute_health_factor(userdata):

    """Computes health factor
    """
    total_borrowed=compute_total_borrowed(userdata)
    total_collateral_threshold=compute_total_collateral_times_thresholds(userdata)
    health_factor = total_collateral_threshold / total_borrowed
    
    return health_factor


def parse_users(queryresult):
  health_factors=[]
  for userdata in queryresult['data']['users']:
    health_factor=compute_health_factor(userdata)
    health_factors.append(health_factor)
  return health_factors


  #for borrow_reserve in user["borrowReserve"]:
    #borrow_reserve["reserve"]["price"]["priceInEth"]

In [ ]:
compute_total_borrowed(userdata0)

0.29762018982300253

In [ ]:
compute_total_collateral_times_thresholds(userdata0)

0.33171635203751176

In [ ]:
compute_health_factor(userdata0)

1.114562665371548

In [ ]:
parse_users(result)

[1.114562665371548,
 1.4270831320454729,
 0.5152897903881213,
 3.7502622037361,
 0.975712874237369,
 3.27830520078925,
 2.056611013952261,
 0.8870117038521624,
 0.22305881374036168,
 0.9986993109665995,
 1.0849235174420002,
 0.5322070223112975,
 8.747071906493892,
 0.7317846556780339,
 1.3910704676104153,
 4.878569249756143,
 1.1478981743370182,
 0.7319254105646756,
 0.3797097386267468,
 1.0773848635998018,
 2.258254825392238,
 0.0597943117284452,
 0.8411752725745409,
 2.0023766339392637,
 3.2523762474579283,
 1.164260714818336,
 2.9271386227121354,
 1.384315163696773,
 1.5736164596087658,
 1.3511645680605973,
 20037.673181970393,
 1.9167175110848291,
 1.7562831736272797,
 2.076296486104553,
 4.390703543364661,
 11.978534705441977,
 1.1254070292469367,
 0.5135330646758275,
 0.8771943046840116,
 1.3514028729049565,
 0.9500047944935858,
 0.5420627079096548,
 0.8978330975334109,
 0.5274123644526371,
 1.0570222804238265,
 2.433623346141777,
 3.644094900243717,
 1.3296028674930358,
 1.30095

In [ ]:
compute_total_borrowed(datauser0)
compute_total_collateral_times_thresholds(datauser0)

In [ ]:
datauser1=result['data']['users'][2]
datauser1

{'id': '0xfffb40347816873ca038756cc87bc610144166b2',
 'borrowedReservesCount': 1,
 'collateralReserve': [{'currentATokenBalance': '1003420809121393',
   'reserve': {'usageAsCollateralEnabled': True,
    'reserveLiquidationThreshold': '8300',
    'reserveLiquidationBonus': '10700',
    'borrowingEnabled': False,
    'utilizationRate': '-0.00003649',
    'symbol': 'STETH',
    'underlyingAsset': '0xae7ab96520de3a18e5e111b5eaab095312d7fe84',
    'price': {'priceInEth': '989207430000000000'},
    'decimals': 18}}],
 'borrowReserve': [{'currentTotalDebt': '1900000',
   'reserve': {'usageAsCollateralEnabled': True,
    'reserveLiquidationThreshold': '8750',
    'borrowingEnabled': True,
    'utilizationRate': '0.43185064',
    'symbol': 'USDC',
    'underlyingAsset': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    'price': {'priceInEth': '841479280000000'},
    'decimals': 6}}]}

In [ ]:
print(compute_total_borrowed(datauser1))
print(compute_total_collateral_times_thresholds(datauser1))
print(compute_health_factor(datauser1))

0.0015988106319999998
0.0008238507954335797
0.5152897903881213


In [ ]:
1/(841479280000000*10**(-18))

1188.38339073542

In [ ]:
def parse_users(payload):
    loans = []

    for user in payload["users"]:
        total_borrowed = 0
        total_collateral = 0
        total_collateral_threshold = 0
        max_borrowed_symbol = None
        max_borrowed_principal = 0
        max_borrowed_price_in_eth = 0
        max_collateral_symbol = None
        max_collateral_bonus = 0
        max_collateral_price_in_eth = 0

        for borrow_reserve in user["borrowReserve"]:
            price_in_eth = borrow_reserve["reserve"]["price"]["priceInEth"]
            principal_borrowed = borrow_reserve["currentTotalDebt"]
            total_borrowed += (
                price_in_eth * principal_borrowed
                / 10 ** borrow_reserve["reserve"]["decimals"]
            )
            if principal_borrowed > max_borrowed_principal:
                max_borrowed_symbol = borrow_reserve["reserve"]["symbol"]
                max_borrowed_principal = principal_borrowed
                max_borrowed_price_in_eth = price_in_eth

        for collateral_reserve in user["collateralReserve"]:
            price_in_eth = collateral_reserve["reserve"]["price"]["priceInEth"]
            principal_a_token_balance = collateral_reserve["currentATokenBalance"]
            total_collateral += (
                price_in_eth * principal_a_token_balance
                / 10 ** collateral_reserve["reserve"]["decimals"]
            )
            total_collateral_threshold += (price_in_eth * principal_a_token_balance * (collateral_reserve['reserve']['reserveLiquidationThreshold'] / 10000)) / 10**collateral_reserve['reserve']['decimals']
            if collateral_reserve['reserve']['reserveLiquidationBonus'] > max_collateral_bonus:
              max_collateral_symbol = collateral_reserve['reserve']['symbol']
              max_collateral_bonus = collateral_reserve['reserve']['reserveLiquidationBonus']
              max_collateral_price_in_eth = price_in_eth

    health_factor = total_collateral_threshold / total_borrowed

    if health_factor <= health_factor_max:
        loans.append({
            "user_id": user['id'],
            "healthFactor": health_factor,
            "max_collateralSymbol": max_collateral_symbol,
            "max_borrowedSymbol": max_borrowed_symbol,
            "max_borrowedPrincipal": max_borrowed_principal,
            "max_borrowedPriceInEth": max_borrowed_price_in_eth,
            "max_collateralBonus": max_collateral_bonus / 10000,
            "max_collateralPriceInEth": max

{'errors': [{'locations': [{'line': 2, 'column': 1}],
   'message': 'Unexpected `users[Name]`\nExpected `{`, `query`, `mutation`, `subscription` or `fragment`'}]}

In [ ]:
    for i, borrow_reserve in enumerate(user['borrowReserve']):
        price_in_eth = borrow_reserve['reserve']['price']['priceInEth']
        principal_borrowed = borrow_reserve['currentTotalDebt']
        total_borrowed += (price_in_eth * principal_borrowed) / 10**borrow_reserve['reserve']['decimals']
        if principal_borrowed > max_borrowed_principal:
            max_borrowed_symbol = borrow_reserve['reserve']['symbol']
            max_borrowed_principal = principal_borrowed
            max_borrowed_price_in_eth = price_in_eth

    for i, collateral_reserve in enumerate(user['collateralReserve']):
        price_in_eth = collateral_reserve['reserve']['price']['priceInEth']
        principal_a_token_balance = collateral_reserve['currentATokenBalance']
        total_collateral += (price_in_eth * principal_a_token_balance) / 10**collateral_reserve['reserve']['decimals']
        total_collateral_threshold += (price_in_eth * principal_a_token_balance * (collateral_reserve['reserve']['reserveLiquidationThreshold'] / 10000)) / 10**collateral_reserve['reserve']['decimals']
        if collateral_reserve['reserve']['reserveLiquidationBonus'] > max_collateral_bonus:
            max_collateral_symbol = collateral_reserve['reserve']['symbol']
            max_collateral_bonus = collateral_reserve['reserve']['reserveLiquidationBonus']
            max_collateral_price_in_eth = price_in_eth

    health_factor = total_collateral_threshold / total_borrowed

    if health_factor <= health_factor_max:
        loans.append({
            "user_id": user['id'],
            "healthFactor": health_factor,
            "max_collateralSymbol": max_collateral_symbol,
            "max_borrowedSymbol": max_borrowed_symbol,
            "max_borrowedPrincipal": max_borrowed_principal,
            "max_borrowedPriceInEth": max_borrowed_price_in_eth,
            "max_collateralBonus": max_collateral_bonus / 10000,
            "max_collateralPriceInEth": max


# Reshape y computo en pandas

In [12]:
import pandas as pd

In [13]:
df = pd.json_normalize(result['data']['users'], sep='_')
df

,id,borrowedReservesCount,collateralReserve,borrowReserve
0,0xffff8941130157a0153fb5be2618b257f28d3b55,1,[{'currentATokenBalance': '385716688415711350'...,"[{'currentTotalDebt': '353684533943004137474',..."
1,0xfffcb3903a402b4669e232b53bc6bcebff6eb131,1,[{'currentATokenBalance': '2573000440777081235...,"[{'currentTotalDebt': '18426617138', 'reserve'..."
2,0xfffb40347816873ca038756cc87bc610144166b2,1,"[{'currentATokenBalance': '1003420809121393', ...","[{'currentTotalDebt': '1900000', 'reserve': {'..."
3,0xfffa889f71003f256f0e14b48e54fa2dad1f2ff0,1,"[{'currentATokenBalance': '1921806252001810', ...","[{'currentTotalDebt': '500000', 'reserve': {'u..."
4,0xfffa5a749b80e9f08ed099909cc9e8244f4beb48,1,"[{'currentATokenBalance': '99999999999999', 'r...","[{'currentTotalDebt': '100000', 'reserve': {'u..."
5,0xfff6cb60f90582e1aaa444aecbd02f21d087edcc,1,[{'currentATokenBalance': '5773868162749661350...,"[{'currentTotalDebt': '1800000000', 'reserve':..."
6,0xfff6b41a29881fc97bd6e8778529b1e37597d94a,1,"[{'currentATokenBalance': '21078035027053598',...","[{'currentTotalDebt': '10000000', 'reserve': {..."
7,0xfff2a069b8af43ad155a72e7579ca2327a8747a8,1,"[{'currentATokenBalance': '9999999999999999', ...","[{'currentTotalDebt': '11000000', 'reserve': {..."
8,0xfff2677022c3b2f53727db9a51b2cce5b4b6631c,1,"[{'currentATokenBalance': '31081505483389472',...","[{'currentTotalDebt': '61166757462003436567', ..."
9,0xffef69efa7f65e233252fdde813ba9f5032e2964,1,[{'currentATokenBalance': '7232693519978899732...,"[{'currentTotalDebt': '205396038974176778758',..."


In [17]:
 json_normalize(result['data']['users'], record_path="borrowReserve", meta=["id"], sep="_")


<ipython-input-17-e2d6287f183b>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  json_normalize(result['data']['users'], record_path="borrowReserve", meta=["id"], sep="_")


,currentTotalDebt,reserve_usageAsCollateralEnabled,reserve_reserveLiquidationThreshold,reserve_borrowingEnabled,reserve_utilizationRate,reserve_symbol,reserve_underlyingAsset,reserve_decimals,reserve_price_priceInEth,id
0,353684533943004137474,True,8700,True,0.31028983,DAI,0x6b175474e89094c44da98b954eedeac495271d0f,18,836885880000000,0xffff8941130157a0153fb5be2618b257f28d3b55
1,18426617138,True,8750,True,0.42855256,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,6,838349457587901,0xfffcb3903a402b4669e232b53bc6bcebff6eb131
2,1900000,True,8750,True,0.42855256,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,6,838349457587901,0xfffb40347816873ca038756cc87bc610144166b2
3,500000,True,8750,True,0.42855256,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,6,838349457587901,0xfffa889f71003f256f0e14b48e54fa2dad1f2ff0
4,100000,True,8750,True,0.42855256,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,6,838349457587901,0xfffa5a749b80e9f08ed099909cc9e8244f4beb48
5,1800000000,True,8750,True,0.42855256,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,6,838349457587901,0xfff6cb60f90582e1aaa444aecbd02f21d087edcc
6,10000000,True,8750,True,0.42855256,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,6,838349457587901,0xfff6b41a29881fc97bd6e8778529b1e37597d94a
7,11000000,True,8750,True,0.42855256,USDC,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,6,838349457587901,0xfff2a069b8af43ad155a72e7579ca2327a8747a8
8,61166757462003436567,False,0,True,0.4391248,BUSD,0x4fabb145d64652a948d72533023f6e7a623c7c53,18,833091981559119,0xfff2677022c3b2f53727db9a51b2cce5b4b6631c
9,68329614858446512,True,8600,True,0.52800523,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,1000000000000000000,0xfff2677022c3b2f53727db9a51b2cce5b4b6631c


In [16]:
 json_normalize(result['data']['users'], record_path="collateralReserve", meta=["id"], sep="_")

<ipython-input-16-f657d229bd78>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  json_normalize(result['data']['users'], record_path="collateralReserve", meta=["id"], sep="_")


,currentATokenBalance,reserve_usageAsCollateralEnabled,reserve_reserveLiquidationThreshold,reserve_reserveLiquidationBonus,reserve_borrowingEnabled,reserve_utilizationRate,reserve_symbol,reserve_underlyingAsset,reserve_decimals,reserve_price_priceInEth,id
0,385716688415711350,True,8600,10500,True,0.52800523,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,1000000000000000000,0xffff8941130157a0153fb5be2618b257f28d3b55
1,25730004407770812351,True,8600,10500,True,0.52800523,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,1000000000000000000,0xfffcb3903a402b4669e232b53bc6bcebff6eb131
2,1003420809121393,True,8300,10700,False,-0.00003649,STETH,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,18,987605093545255600,0xfffb40347816873ca038756cc87bc610144166b2
3,1921806252001810,True,8300,10700,False,-0.00003649,STETH,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,18,987605093545255600,0xfffa889f71003f256f0e14b48e54fa2dad1f2ff0
4,99999999999999,True,8300,10700,False,-0.00003649,STETH,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,18,987605093545255600,0xfffa5a749b80e9f08ed099909cc9e8244f4beb48
5,5773868162749661350,True,8600,10500,True,0.52800523,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,1000000000000000000,0xfff6cb60f90582e1aaa444aecbd02f21d087edcc
6,21078035027053598,True,8300,10700,False,-0.00003649,STETH,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,18,987605093545255600,0xfff6b41a29881fc97bd6e8778529b1e37597d94a
7,9999999999999999,True,8300,10700,False,-0.00003649,STETH,0xae7ab96520de3a18e5e111b5eaab095312d7fe84,18,987605093545255600,0xfff2a069b8af43ad155a72e7579ca2327a8747a8
8,31081505483389472,True,8600,10500,True,0.52800523,WETH,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,18,1000000000000000000,0xfff2677022c3b2f53727db9a51b2cce5b4b6631c
9,7232693519978899732,True,6200,10750,False,0.63500837,SNX,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,18,1226311100000000,0xffef69efa7f65e233252fdde813ba9f5032e2964


In [65]:
def compute_total_collateral_times_thresholds():
  df=json_normalize(result['data']['users'], record_path="collateralReserve", meta=["id"], sep="_")

  df['price_in_eth'] = df.reserve_price_priceInEth.astype('int64')/ (10 ** 18)
  df['decimals']=df.reserve_decimals.astype('int64')
  df['current_a_token_balance'] = df["currentATokenBalance"].astype('float')/ (10 ** df.decimals)
  df['aux']=df['price_in_eth'] * df['current_a_token_balance']

  df2=df.groupby(['id']).sum(['aux'])


  #liquidation_threshold=(int(collateral_reserve['reserve']['reserveLiquidationThreshold']) / 10000)
  #sum of collateral times thresholds 
  #total_collateral_threshold += price_in_eth * current_a_token_balance * liquidation_threshold
  return df2

 
compute_total_collateral_times_thresholds()



<ipython-input-65-76e3044c32f8>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df=json_normalize(result['data']['users'], record_path="collateralReserve", meta=["id"], sep="_")


,reserve_usageAsCollateralEnabled,reserve_borrowingEnabled,reserve_decimals,price_in_eth,decimals,current_a_token_balance,aux
id,,,,,,,
0xffef69efa7f65e233252fdde813ba9f5032e2964,2,1,36,1.001226,36,7.643349,0.419525
0xfff2677022c3b2f53727db9a51b2cce5b4b6631c,1,1,18,1.000000,18,0.031082,0.031082
0xfff2a069b8af43ad155a72e7579ca2327a8747a8,1,0,18,0.987605,18,0.010000,0.009876
0xfff6b41a29881fc97bd6e8778529b1e37597d94a,1,0,18,0.987605,18,0.021078,0.020817
0xfff6cb60f90582e1aaa444aecbd02f21d087edcc,1,1,18,1.000000,18,5.773868,5.773868
0xfffa5a749b80e9f08ed099909cc9e8244f4beb48,1,0,18,0.987605,18,0.000100,0.000099
0xfffa889f71003f256f0e14b48e54fa2dad1f2ff0,1,0,18,0.987605,18,0.001922,0.001898
0xfffb40347816873ca038756cc87bc610144166b2,1,0,18,0.987605,18,0.001003,0.000991
0xfffcb3903a402b4669e232b53bc6bcebff6eb131,1,1,18,1.000000,18,25.730004,25.730004
